In [ ]:
# Default analysis (non-extended source)
# for a spherical diffuser tip

In [ ]:
import numpy as np
from tables import get_c5, table3, table_a1, table_a5

In [ ]:
# Note, all time values are expressed in seconds [s]
# The analysis is for a repetitively pulsed laser
# emitting at a single wavelength

t_pulse = 80e-15  # pulse duration [s]
lambda_ = 784  # wavelength [nm]
f = 80.1e6  # laser rep rate [Hz]

# TODO make automatic determination based on class (and wavelength?)
t_base = 100  # time base for Class 1

In [ ]:
## AEL

In [ ]:
### Single pulse analysis (determining AEL_single)
ael_single_j = table3(lambda_, t_pulse)[0]

In [ ]:
### Average power analysis 
ael_t_w = table3(lambda_, t_base)[1]
# see Note 8 of 4.3.f and Note 1 in Figure B.1
ael_spt_j = ael_t_w / f

In [ ]:
### Reduced pulse analysis (pulse train analysis) 
# get correction factor c5, the effective pulse duration,
# and the number of real pulses in the eff. pulse duration
# Note, alpha is kept at alpha_min for default evaluation
c5, t_eff, n = get_c5(lambda_, t_pulse, 1 / f, t_base, 1.5e-3)

# single pulse AEL for the effective pulse length [J]
ael_eff_j = table3(lambda_, t_eff)[0]
# project back to an individual pulse and apply correction
ael_sptrain_j = ael_eff_j / n * c5

In [ ]:
ael = min(ael_single_j, ael_spt_j, ael_sptrain_j)
ael

In [ ]:
## Accessible emission

In [ ]:
p_avg = 0.124  # average laser power
# duty = t_pulse * f  # duty cycle of laser pulses
q_pulse = p_avg / f  # pulse energy


# Function to get energy incident on aperture
def ae(q, w, r):
    """
    Accessible emission from spherical diffuser
    :param q: pulse energy [J]
    :param w: aperture diameter [m]
    :param r: aperture distance from sphere surface [m]
    :return: accessible emission [J]
    """
    r_c = r + 1e-3
    theta = np.atan(w / 2 / r_c)
    # pulse energy on aperture
    return q * (1 - np.cos(theta)) / 2

In [ ]:
### Condition 1

In [ ]:
# Condition 1
# Just in case (to see if class is actually 1M)
w_c1 = 50e-3  # aperture diameter (Table 10)
d_c1 = 2000e-3  # distance (Table 10, 11)
q_single_c1 = ae(q_pulse, w_c1, d_c1)

In [ ]:
### Condition 3

In [ ]:
w_c3 = 7e-3  # aperture diameter (Table 10)
d_c3 = 100e-3  # distance (Table 10, 11)
# accessible emission (in single pulse) [J]
q_single_c3 = ae(q_pulse, w_c3, d_c3)

In [ ]:
# Check class
# q_ap_single_c1, q_ap_single_c3
if q_single_c3 <= ael:
    if q_single_c1 <= ael:
        print('Product is Class 1!')
    else:  # q_ap_single_c1 > ael, technically, we should check if it's also below class 3B
        print('Product is Class 1M')
else:
    print('Product is above Class 1, repeat test with higher class specs')


In [ ]:
## MPE (eye)

In [ ]:
### Single pulse analysis
mpe_single_eye = table_a1(lambda_, t_pulse)[0]

In [ ]:
### Average power analysis 
mpe_avg_eye_i = table_a1(lambda_, t_base)[1]
# see Note 8 of 4.3.f and Note 1 in Figure B.1
mpe_avg_eye_re = mpe_avg_eye_i / f

In [ ]:
### Reduced pulse analysis (pulse train analysis) 
# use correction factor c5, the effective pulse duration,
# and the number of real pulses in the eff. pulse duration

# single pulse AEL for the effective pulse length [J]
mpe_eff_re = table_a1(lambda_, t_eff)[0]
# project back to an individual pulse and apply correction
mpe_sptrain_re = mpe_eff_re / n * c5

In [ ]:
mpe_eye = min(mpe_single_eye, mpe_avg_eye_re, mpe_sptrain_re)
mpe_eye

In [ ]:
# of course, we should get the same if using class 1 AEL
# calculated already, and divide it with the aperture area
def get_area(w):
    """
    :param w: aperture diameter [m] 
    :return: aperture area [m2]
    """
    return (w / 2) ** 2 * np.pi


a_ap_eye = get_area(w_c3)  # aperture area for eye
mpe_2 = ael / a_ap_eye
mpe_2

In [ ]:
## MPE (skin)

In [ ]:
### Single pulse analysis
mpe_simple_skin_re = table_a5(lambda_, t_pulse)[0]

In [ ]:
### Average power analysis 
mpe_avg_skin_i = table_a5(lambda_, t_base)[1]
# see Note 8 of 4.3.f and Note 1 in Figure B.1
mpe_avg_skin_re = mpe_avg_skin_i / f

mpe_skin = min(mpe_simple_skin_re, mpe_avg_skin_re)

In [ ]:
## NOHD 
# array of aperture distances 
ds = np.arange(1e-4, 150e-3, 1e-4)
# accessible emission values at distance d
# (ae is vectorised, so can call it with array)
q_d = ae(q_pulse, w_c3, ds)
# array of radiant exposure values
re_eye = q_d / a_ap_eye
nohd = ds[np.where(re_eye <= mpe_eye)][0]
nohd

In [ ]:
# As a sanity check, we could use the Tools / Diff. Refl. tab in LaserBee,
# with a 'Point Emitter' model.
# It only calculates the MPE for CW but that could be directly compared
# with our average MPE calculated for a pulse train.
# For the default emission time (T_base), and evaluation position (100 mm)
# This should be 14.72 W/m2, according to LaserBee
mpe_avg_eye_i

In [ ]:
# Similarly, we can the radiant energy for the default conditions
# According to LaserBee, this should give a radiant exposure of 
# 98.678 J/m2
q_d = ae(p_avg * t_base, w_c3, d_c3)
q_d / a_ap_eye

In [ ]:
## Skin hazard distance
w_skin = 3.5e-3  # aperture diameter for skin [m]
a_ap_skin = get_area(w_skin)
# accessible emission values at distance d
# (ae is vectorised, so can call it with array)
q_d = ae(q_pulse, w_skin, ds)
# array of radiant exposure values
re_skin = q_d / a_ap_skin
shd = ds[np.where(re_skin <= mpe_skin)][0]
shd